# PDF Insight Navigator

Install required libraries:

In [1]:
!pip install langchain tiktoken faiss-cpu gpt4all pypdf huggingface-hub InstructorEmbedding sentence_transformers vectordb

Download the Mistral-Instruct model:

In [7]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q5_K_M.gguf -P models

--2024-01-19 17:40:23--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.238.49.112, 18.238.49.70, 18.238.49.117, ...
Connecting to huggingface.co (huggingface.co)|18.238.49.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/c4b062ec7f0f160e848a0e34c4e291b9e39b3fc60df5b201c038e7064dbbdcdc?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q5_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q5_K_M.gguf%22%3B&Expires=1705945223&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTk0NTIyM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2L2M0YjA2MmVjN

Example implementation of Minstral 7b using GPT4ALL

In [3]:
from gpt4all import GPT4All
model = GPT4All("mistral-7b-openorca.Q4_0.gguf")
output = model.generate("The capital of France is ", max_tokens=200, temp=0.7, top_k=40, top_p=0.4, repeat_penalty=1.18, repeat_last_n=64, n_batch=8, n_predict=None, streaming=False)
print(output)

2,000 years old and has a rich history. It’s the largest city in France with over 10 million people living within its metropolitan area. Paris is known for many things including its iconic Eiffel Tower, Notre Dame Cathedral, Louvre Museum, and the River Seine that runs through it.



Imports :

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import gpt4all
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager
from langchain.prompts import PromptTemplate
import vectordb

## Data Loading and Preparation:

In [3]:
# Load and split PDF documents:
documents = PyPDFLoader('/content/NIPS-2017-attention-is-all-you-need-Paper.pdf').load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=64)
texts = text_splitter.split_documents(documents)

# Create vector database:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
vectordbs = FAISS.from_documents(texts, instructor_embeddings)
vectordbs.save_local("database")
vectordbs = FAISS.load_local("database", instructor_embeddings)

load INSTRUCTOR_Transformer
max_seq_length  512


## Model Configuration:

In [4]:
# Load LLM and create prompt template:
question = "What are transformers?"
matched_docs = vectordbs.similarity_search(question, 4)
context = ""
for doc in matched_docs:
  context = context + doc.page_content + " \n\n "
print(context)

template = """
Please use the following context to answer questions.
Context: {context}
 - -
Question: {question}
Answer: Let's think step by step."""

callback_manager = BaseCallbackManager([StreamingStdOutCallbackHandler()])
llm = GPT4All(model='models/mistral-7b-instruct-v0.1.Q5_K_M.gguf', callback_manager=callback_manager, verbose=True)
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
llm_chain = LLMChain(prompt=prompt, llm=llm)

language modeling tasks [28].
To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying
entirely on self-attention to compute representations of its input and output without using sequence-
aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [14, 15] and [8].
3 Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure [ 5,2,29].
Here, the encoder maps an input sequence of symbol representations (x1,...,x n)to a sequence
of continuous representations z= (z1,...,z n). Given z, the decoder then generates an output
sequence (y1,...,y m)of symbols one element at a time. At each step the model is auto-regressive
[9], consuming the previously generated symbols as additional input when generating the next.
The Transformer follows this overall architecture using stacked self-attention and point-wise, full

In [5]:
query = "What are transformers?"
docs = vectordbs.similarity_search(query)
docs[0]

Document(page_content='language modeling tasks [28].\nTo the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying\nentirely on self-attention to compute representations of its input and output without using sequence-\naligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate\nself-attention and discuss its advantages over models such as [14, 15] and [8].\n3 Model Architecture\nMost competitive neural sequence transduction models have an encoder-decoder structure [ 5,2,29].\nHere, the encoder maps an input sequence of symbol representations (x1,...,x n)to a sequence\nof continuous representations z= (z1,...,z n). Given z, the decoder then generates an output\nsequence (y1,...,y m)of symbols one element at a time. At each step the model is auto-regressive\n[9], consuming the previously generated symbols as additional input when generating the next.\nThe Transformer follows this overall architecture using stacked s

In [6]:
# Generate an answer using retrieved context:
question = 'What are transformers?'
print(llm_chain.invoke(question))

 Transformers are a type of neural network architecture that is used for sequence transduction tasks, such as language modeling and machine translation. They were first introduced in the paper "Attention Is All You Need" [28]. The main idea behind transformers is to use self-attention instead of recurrent networks or convolution to compute representations of input and output sequences. This allows for more parallelization and can lead to better performance on certain tasks, such as machine translation.

Transformers have a stacked architecture with two sub-layers in each layer: multi-head self-attention and point-wise fully connected layers. The encoder maps an input sequence of symbol representations to a sequence of continuous representations, while the decoder generates an output sequence of symbols one element at a time, consuming the previously generated symbols as additional input when generating the next.

The Transformer architecture is shown in Figure 1. It consists of two ide